# Project: Dogs Vs Cats classification.
* This is Part 2
* Using a pre-trained convent CNN

* part 1 and details: https://github.com/DevHwary/CNN-CatsVsDogs

# 1- Data Preparing

In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator

# Data is prepared, only get the directories of the data
base_dir = '/home/hwary/Deep Learning With Python/5-DogsVsCats/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


# Data Augmentation for only the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


Using TensorFlow backend.


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# 2- Instantiating the VGG16 convolutional base
* By using the convolutional base of the VGG16 network, trained on ImageNet, to extract interesting features from cat and dog images, and then train a dogs-versus-cats classifier on top of these features.


In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

# 3- Fine-tuning
Fine-tuning consists of unfreezing a few of the top layers of a frozen model base used for feature extraction, and jointly training both the newly added part of the model (in this case, the fully connected classifier) and these top layers.

fine-tune the last three convolutional layers, which means all layers up to block4_pool should be frozen, and the layers block5_conv1, block5_conv2, and block5_conv3 should be trainable

In [3]:
# Freezing all layers up to a specific one

conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [4]:
#  Adding a densely connected classifier on top of the convolutional base
from keras import models, layers, optimizers

model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))      
model.add(layers.Dropout(0.5))          # the dropout layer fighting the overfitting 
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [5]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])


In [6]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50)


Epoch 1/20
100/100 [==============================] - 803s 8s/step - loss: 0.6465 - acc: 0.6235 - val_loss: 0.4321 - val_acc: 0.8150
Epoch 2/20
100/100 [==============================] - 807s 8s/step - loss: 0.4754 - acc: 0.7730 - val_loss: 0.2960 - val_acc: 0.8710
Epoch 3/20
100/100 [==============================] - 806s 8s/step - loss: 0.3918 - acc: 0.8150 - val_loss: 0.2400 - val_acc: 0.8980
Epoch 4/20
100/100 [==============================] - 803s 8s/step - loss: 0.3485 - acc: 0.8440 - val_loss: 0.3006 - val_acc: 0.8660
Epoch 5/20
100/100 [==============================] - 804s 8s/step - loss: 0.3381 - acc: 0.8590 - val_loss: 0.2114 - val_acc: 0.9100
Epoch 6/20
100/100 [==============================] - 808s 8s/step - loss: 0.3098 - acc: 0.8640 - val_loss: 0.1935 - val_acc: 0.9140
Epoch 7/20
100/100 [==============================] - 806s 8s/step - loss: 0.2687 - acc: 0.8875 - val_loss: 0.1981 - val_acc: 0.9110
Epoch 8/20
100/100 [==============================] - 805s 8s/step - 

In [7]:
# saving the model number 2

model.save('cats_and_dogs_small_3.h5')

In [8]:
# Displaying curves of loss and accuracy during training
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

part Two is finished
* Acc: 94%
* Loss: 16%
* Val_acc: 92%
* Val_loss: 20%

we hade fighted the overfitting and tuned the model well.

Note : if we make epochs=100 not only 20, we would reach accuracy equal 97%. But i have no GPU right now.

# Evaluate the model on the test data.

In [11]:
test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary')


test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test accuracy: ', test_acc)

Found 1000 images belonging to 2 classes.
('test accuracy: ', 0.9229999935626984)


test accuracy would reach 97% too if we have finished the 100 epochs.
it is Great! Woa

# In part 3: Visualizing what convents 'CNN' learn